# installs requirements

In [1]:
!pip install gradio
!pip install fsspec --upgrade
!pip install pandas

  Obtaining dependency information for gradio from https://files.pythonhosted.org/packages/01/12/db61f50001a9c7e0a6465543435b28d16c7fcbde8ca58f31f5a5657de203/gradio-4.2.0-py3-none-any.whl.metadata
  Obtaining dependency information for aiofiles<24.0,>=22.0 from https://files.pythonhosted.org/packages/c5/19/5af6804c4cc0fed83f47bff6e413a98a36618e7d40185cd36e69737f3b0e/aiofiles-23.2.1-py3-none-any.whl.metadata
  Using cached aiofiles-23.2.1-py3-none-any.whl.metadata (9.7 kB)
  Obtaining dependency information for altair<6.0,>=4.2.0 from https://files.pythonhosted.org/packages/17/16/b12fca347ff9d062e3c44ad9641d2ec50364570a059f3078ada3a5119d7a/altair-5.1.2-py3-none-any.whl.metadata
  Using cached altair-5.1.2-py3-none-any.whl.metadata (8.6 kB)
  Obtaining dependency information for fastapi from https://files.pythonhosted.org/packages/f3/4f/0ce34195b63240b6693086496c9bab4ef23999112184399a3e88854c7674/fastapi-0.104.1-py3-none-any.whl.metadata
  Obtaining dependency information for gradio-clie

ERROR: Could not install packages due to an OSError: [WinError 2] The system cannot find the file specified: 'C:\\Python311\\Scripts\\pygmentize.exe' -> 'C:\\Python311\\Scripts\\pygmentize.exe.deleteme'


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


  Obtaining dependency information for fsspec from https://files.pythonhosted.org/packages/e8/f6/3eccfb530aac90ad1301c582da228e4763f19e719ac8200752a4841b0b2d/fsspec-2023.10.0-py3-none-any.whl.metadata
  Using cached fsspec-2023.10.0-py3-none-any.whl.metadata (6.8 kB)
Using cached fsspec-2023.10.0-py3-none-any.whl (166 kB)



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


  Obtaining dependency information for pandas from https://files.pythonhosted.org/packages/97/d8/dc2f6bff06a799a5603c414afc6de39c6351fe34892d50b6a077df3be6ac/pandas-2.1.3-cp311-cp311-win_amd64.whl.metadata
  Using cached pandas-2.1.3-cp311-cp311-win_amd64.whl.metadata (18 kB)
Using cached pandas-2.1.3-cp311-cp311-win_amd64.whl (10.6 MB)



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# Imports
import pandas as pd
import gradio as gr
import sqlite3

In [3]:
%%bash
# Connects to the Server and Runs the last command
#!/usr/bin/bash
#How to install sshpass package
#last -s yesterday -t today
sshpass -f password.txt ssh user@IP ADDRESS
cd Desktop/F_Jaramillo_Project
output="$(last -s yesterday -t today)"
echo "$output" >>log.csv

-bash: line 5: sshpass: command not found
-bash: line 6: cd: Desktop/F_Jaramillo_Project: No such file or directory


In [4]:
names = ['Username', 'IP', 'LoginTime', 'LogoutTime', 'TotalTime', 'Terminal']
df = pd.read_csv('log.csv', sep=' ', names=names)
print(df.head())

     Username   IP LoginTime  LogoutTime TotalTime  Terminal
wtmp   begins  Sat       Oct          21  16:28:14      2023


In [5]:
#Creates the Table in SQLite
import sqlite3
conn = sqlite3.connect("test.db")
cur = conn.cursor()

sql = """
create table Table1
    (Username varchar(256),
    IP varchar(256),
    LoginTime varchar(256),
    LogoutTime varchar(256),
    TotalTime varchar(256),
    Terminal varchar(256)
    );
"""

cur.execute(sql)
print("database has been created")

conn.commit()
conn.close()

database has been created


In [6]:
import sqlite3
conn = sqlite3.connect("test.db")
df.to_sql("Table1", conn,schema=None, if_exists='append', index=False, index_label=None, chunksize=6, dtype=None, method=None)
conn.close()

In [7]:
#Converts the Database to a pandas DataFrame so it can be used on the Gradio web page
connection = sqlite3.connect("test.db")
sql = pd.read_sql_query("SELECT * FROM Table1", connection)
df = pd.DataFrame(sql, columns=["Username", "IP","LoginTime", "LogoutTime",
                     "TotalTime","Terminal"])
connection.close()
print(df)

  Username   IP LoginTime LogoutTime TotalTime Terminal
0   begins  Sat       Oct         21  16:28:14     2023


In [31]:
#Creates the Gradio webpage
import gradio as gr

css = """
#warning textarea {background-color: #595cff;
}
"""

def filter_records(records, Username):
    return records[records["Username"] == Username]


with gr.Blocks(theme=gr.themes.Base(), css=css) as first_page:
    with gr.Row():
        Textbox=gr.Textbox(scale=0, show_label=False, value='Look up', elem_id="warning")
        gr.Textbox(scale=2, show_label=False)
        gr.Textbox(scale=0, show_label=False, value='in', elem_id="warning")
        dropdown=gr.Dropdown(["Username", "IP Address","Login Time", "Logout time",
                     "Total Time","Terminal"],
                     scale=2, show_label=False)
        button = gr.Button(value="Search", size='sm', scale=0)
    with gr.Row():
        df_info = gr.Dataframe(height=800, value=df,
                     headers=["Username", "IP Address","Login Time", "Logout time",
                     "Total Time","Terminal"],
                     datatype=["str", "str", "str", "str", "str", "str"],
                     row_count=10,
                     col_count=(6, "fixed"))
    gr.Label=()
first_page.launch(share=True)

Running on local URL:  http://127.0.0.1:7873
Running on public URL: https://a7244015a9b3f0a392.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


# App for monitoring SSH connections

SSH monitoring is not very user friendly because it needs to be done from a terminal with different commands and requires a connection to be made using a username, IP address, and password. This is a software that will make monitoring an ssh server more simple. I created this software to make it more user friendly with a nicer-looking interface and putting all the data in one place. Monitoring an SSH server is easier with this software as it will get the information from the server send it to a database and display it in a website for the user. The languages used to create this software were Python, Bash, and sqlite3. 

  

The purpose of this software is to monitor a server in the most efficient way. It monitors the server remotely and the user does not need to run the commands themselves because the bash script automatically connects and runs the commands needed monitor. All data from the commands being ran will be saved to a database and this will be displayed on a website that is very simple and user friendly. With a cron job set up this notebook will be ran automatically each day and saved to the database. 

  

Some of the key features this software has are stores data in 1 place, constant updates to the database, a GUI, and automatic connection to the ssh server. Connection to a server is automatic for the current server is connected to. If the user wishes to change the server this can be done by changing cell 3s username, IP, and password. The database is updated every 24 hours using a cron job which runs the jupyter notebook at 11:59pm each day. The GUI was created using the Gradio library in python and has the ability to sort items and search for items inside of the database. All data is first stored into a file and then sent to the database which is then displayed on the Gradio GUI. 

  

Firstly when creating this project me and my mentor brainstormed some ideas for the app and settled on which languages were going to be used when creating it. Our main goals for the app were to create something that could log a users current session and save it to a file which would send that to a databasea at a certain time each day. We set goals that I needed to reach each week while working on this app. The way I would reach these goals is by doing research on the task I had each week. The tools I used were youtube, geeksforgeeks.com, and pandas.pydata.org. The order in which I created the app was the database first, the GUI, the bash script and the pandas data frame. 

  

The benefit of this software is that it makes monitoring more efficient by giving the user access to the information to everything needed to monitor a server. The look of the GUI makes it more user friendly so a user does not need to know how to run command lines and connect to an ssh server since it is all done for the user. With the GUI finding data is simple as there is a search bar and a way to sort items within a column. Another benefit is all data is stored in a database so older records can be looked at. 

  

The main goal of this project was to create a software that makes it easier for a user to monitor a server. I think the goal was reached with this software as it allows for someone to monitor the server remotely without needing to run commands in the terminal. With the gradio link the user monitoring can view past records of users who have logged in. 